In [1]:
# Jupyter notebook to pull rwb data and run analysis on it.
# Ensure that you are connected to the Tetramem network to pull data.
# fuck you
import pandas as pd
import numpy as np
import sys, os, re, math
import matplotlib.pyplot as plt
import scipy.stats as stats
import subprocess
import json
# Insert path to "flint_mpw4_testing\tests\postprocess" if using rwb_viewer notebook outside of this folder
sys.path.insert(1, r"C:\Users\AdrienPierre\Documents\flint_mpw4_testing\tests\postprocess")
import core_post_processing_functions as cf
import create_and_upload_rwb_param as curp

ModuleNotFoundError: No module named 'core_post_processing_functions'

This is an example analysis of correlating pre-bake vs. post-bake metrics. The first half will look at IO-level correlations while the second half of teh analysis will explore bit-level correlations. It is highly recommended to install Data Wrangler extension in VS Code for easy viewing of pandas dataframes. After installing this extension a button called 'View data' should appear at the top bar of your jupyter notebook.

1. IO-level correlations

In [2]:
# Pull all data from TT27
rwb_pull = cf.rwb_fetch_data(regex='TT27', regex_col='DIE_ID') # Leave empty to pull all RWB data, takes longer time

NameError: name 'cf' is not defined

In [3]:
# Determine which macros/IOs were baked
baked_tt27macroios = rwb_pull.loc[rwb_pull.RUN_NAME.str.contains('bake')][['DIE_ID','MACRO','IO']].drop_duplicates() # drop duplicates is to remove duplicate items after only keeping die, macro and IO columns

# Determine which macros/IOs were cycled up to 100
cyc100_tt27macroios = rwb_pull.loc[rwb_pull.TEST_NAME.str.contains('100cyc')][['DIE_ID','MACRO','IO']].drop_duplicates()

# Find the intersection of the two
RAC_tt27macroios = pd.merge(left=baked_tt27macroios, right=cyc100_tt27macroios, how='inner') # Use inner intersection (default) to keep common macros/IOs for these two conditions (baked and cycled)
print(RAC_tt27macroios)


NameError: name 'rwb_pull' is not defined

In [4]:
# Filter your pulled data to the found die by doing an inner merge
rwb_RAC = pd.merge(left=rwb_pull, right=RAC_tt27macroios, how='inner')

# Print the unique TEST_NAMEs to sanity check
test_name_counts = rwb_RAC.TEST_NAME.value_counts() # Value counts shows the number of rows for each unique value
print(test_name_counts.to_string()) # .to_string() is added at the end to enable full viewing of this list

NameError: name 'rwb_pull' is not defined

In [ ]:
# Let's only keep data for tests on all 32 measured IOs
tests_of_interest = test_name_counts[test_name_counts >= 32].index.to_list()
tests_of_interest

In [ ]:
# The TEST_NAME 'T0_read0' has 36 readouts even though there are only 32 IOs, let's find out where the redundancy in the data is
rwb_T0debug = rwb_RAC.loc[rwb_RAC.TEST_NAME=='T0_read0']

# Using Data Wranger to explore the data frame, we notice that all duplicate runs come from what looks like a validation run on IO0 before KPI called 'IO0preKPI'. TO be safe let's exclude IO0 from our study
rwb_RAC = rwb_RAC.loc[~(rwb_RAC.IO==0)]

In [ ]:
# The updated TEST_NAMEs are shown below
test_name_counts = rwb_RAC.TEST_NAME.value_counts() # Value counts shows the number of rows for each unique value
print(test_name_counts.to_string()) # .to_string() is added at the end to enable full viewing of this list

It looks like the dataset is cleaned up, let's start plotting!

In [ ]:
# Plot the NQ distributions, let's say we want to check out the 3 IOs with the best (widest) and worst (narrowest) T0 read window for Level 0/1 just to gauge the data.

# Locate T0 readout and sort rows based on level0/1 windwo at 170PPM. Higher row index (further down the table) means worse window.
rwb_T0_sorted_by_level01window = rwb_RAC.loc[rwb_RAC.TEST_NAME=='T0_read0'].sort_values(by='LEVEL_01_170PPM_ADCRWB', ascending=False)

print('The best 3 IOs for LEVEL_01_170PPM_ADCRWB')
cf.rwb_overlay_levels_nqplot(rwb_T0_sorted_by_level01window.iloc[:3], groupby_col='IO')

print('The worst 3 IOs for LEVEL_01_170PPM_ADCRWB')
cf.rwb_overlay_levels_nqplot(rwb_T0_sorted_by_level01window.iloc[-3:], groupby_col='IO')

In [ ]:
# We can also plot the same data overlaying the previously plotted IOs grouped by level

# Find the best and worst 3 IOs
IOs = list(rwb_T0_sorted_by_level01window.iloc[:3].IO) + list(rwb_T0_sorted_by_level01window.iloc[-3:].IO)

# Generate plot
cf.rwb_groupby_level_nqplot(rwb_T0_sorted_by_level01window.loc[rwb_T0_sorted_by_level01window.IO.isin(IOs)]) 

One can see from the figure above that the main differentiator for level 0/1 window is the tail formation in level1

In [ ]:
# Now that we have a rough handle for the appearance of the raw data, we can now examine IO-level correlations. Let's first look at RAC vs. the same T0 window metric

# This function pivots the RWB table by TEST_NAME by grouping die/macro/IO information and splitting all data columns by unique TEST_NAMEs
rwb_RAC_split = cf.rwb_pivot_by_test(rwb_RAC)

# Plot scatter plots of various T0 metrics vs. post-bake metrics to explore which ones generate the best fit
cf.rwb_scatter_plot(rwb_RAC_split, x='T0_read0 - LEVEL_01_XPOINT_PPM', y='150C220minbake_read0 - LEVEL_01_XPOINT_PPM', overlay_var='DIE_ID', fit_type='linear')
cf.rwb_scatter_plot(rwb_RAC_split, x='T0_read0 - LEVEL_01_170PPM_ADCRWB', y='150C220minbake_read0 - LEVEL_01_XPOINT_PPM', overlay_var='DIE_ID', fit_type='linear')
cf.rwb_scatter_plot(rwb_RAC_split, x='T0_read0 - LEVEL_0_0ADC_PPM', y='150C220minbake_read0 - LEVEL_01_XPOINT_PPM', overlay_var='DIE_ID', fit_type='linear')
cf.rwb_scatter_plot(rwb_RAC_split, x='T0_read0 - LEVEL_1_DISTSIGMA', y='150C220minbake_read0 - LEVEL_01_XPOINT_PPM', overlay_var='DIE_ID', fit_type='linear')

2. Bit-level data

In [ ]:
# The follow code snippet it used to fetch bit-level data for further deep dive. For isntance let's pull the bit level data for the 3 best and 3 worst IOs we found earlier

# Create a dataframe of just the rows of interest, don't pull too much data (no more than 20 unique readouts) since it will be slow to process
rwb_bestandworstIOs = rwb_RAC.loc[(rwb_RAC.IO.isin([35,2]))&(rwb_RAC.TEST_NAME.isin(['T0_read0','pst100cycft_read0','150C220minbake_read0']))]

# Fetch the raw bit-level data, each row is a unique die/macro/io/BL/WL and each coulmn is grouped by TEST_NAME  
df_bit_level = cf.rwb_fetch_bitlevel_data(rwb_bestandworstIOs)

In [ ]:
# View the first ten rows of bit level data
df_bit_level.head(10)

In [ ]:
# Save data to csv file for further analysis
df_bit_level.to_csv('bit_level_data.csv', index=False)

In [ ]:
# I haven't gotten a chance to write functions that plot bit-level data in python yet, currently using JMP for this type of analysis